In [3]:
import os
import re
import pandas as pd
from deep_translator import (GoogleTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator)
import string 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob
import plotly.express as px

## Importing Data

In [4]:
def remove_unnnamed(data):
    data = data.drop(columns=['Unnamed: 0'])
    return data

In [5]:
def remove_emojis(df):
    #df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
    if "Translation" in df:
        res = df[df['Translation'].apply(lambda x: not isinstance(x, float))]
    else:
        res = df[df['content'].apply(lambda x: not isinstance(x, float))]
    return res

In [6]:
def remove_emtpy_translation(df):
    if "Translation" in df:
        df = df.dropna(subset=['Translation'])
    else:
        df = df.dropna(subset=['content'])
    return df

Scraping, 100 results so far
Scraping, 200 results so far
Scraping, 300 results so far
Scraping, 400 results so far
Scraping, 500 results so far
Stopped scraping after 500 results due to --max-results


KeyboardInterrupt: 

In [474]:
pres = {}

In [7]:
def remove_url(s):
    try:
        s = re.sub(r'http\S+', '', s)
        return s
    except:
        return s

FileNotFoundError: [Errno 2] No such file or directory: 'EmmanuelMacron_tweets.csv'

In [ ]:
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in string.punctuation])
    return s

In [ ]:
def lower_case(s):
    s = s.lower()
    return s

In [ ]:
def remove_numbers(s):
    s = ''.join(word for word in s if not word.isdigit())
    return s

In [ ]:
def lemmatize(s):
    lemmatizer = WordNetLemmatizer()
    s = ''.join(lemmatizer.lemmatize(word) for word in s)
    return s

In [ ]:
def stop_words(s):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(s) 
    s = ' '.join(w for w in word_tokens if not w in stop_words)
    return s

In [ ]:
#Cleaning the strings
for name, data in tweets.items():
    if "Translation" in data:
        data['clean_content'] = data['Translation'].apply(remove_url).apply(remove_punctuation).\
        apply(lower_case).apply(remove_numbers).apply(stop_words).apply(lemmatize)
    else:
        data['clean_content'] = data['content'].apply(remove_url).apply(remove_punctuation).\
        apply(lower_case).apply(remove_numbers).apply(stop_words).apply(lemmatize)
    tweets[name] = data

## Textblob Sentiment Analysis

In [ ]:
def polarity(s):
    pol = TextBlob(s)
    pol = pol.sentiment[0]
    return pol

def objectivity(s):
    obj = TextBlob(s)
    obj = obj.sentiment[1]
    return obj

In [ ]:
sentiment_df = {'polarity': [list(sentiment.values())[i]['polarity'].mean() for i in range(0,len(sentiment))],\
                'objectivity': [list(sentiment.values())[i]['objectivity'].mean() for i in range(0,len(sentiment))]}

sentiment_df = pd.DataFrame(data=sentiment_df, index=sentiment.keys())
sentiment_df

In [ ]:
fig = px.scatter(sentiment_df, x="polarity", y="objectivity", color=sentiment_df.index)
fig.show()

In [ ]:
## LDA

In [669]:
def lda_model(tweet_dict):
    for i in range(0, len(tweet_dict)):
        for presname, dataframe in list(tweet_dict.items())[i]:
            vectorizer = TfidfVectorizer().fit(dataframe['clean_content'])
            data_vectorized = vectorizer.transform(dataframe['clean_content'])
            model = LatentDirichletAllocation(n_components=3).fit(data_vectorized)
    return model

In [ ]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

In [ ]:
list(tweets.items())[0]